In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import json

In [7]:
url = 'https://store.steampowered.com/search/results/?query&start=1&count=300&dynamic_data=&sort_by=_ASC&tags=3843&untags=4182&category1=998&category3=36%2C9%2C38%2C1&os=win&filter=globaltopsellers&infinite=1'

In [8]:
response = requests.get(url) #check if page is accessible
response

<Response [200]>

In [7]:
def get_rating(game):
    #gets game rating through parsing BS tree
    
    try:
        review_full = game.find(attrs={"class": "search_review_summary"})
        output = review_full['data-tooltip-html']
        return float(output.split('<br>')[1].split('%')[0])
    except TypeError:
        return '-1.0'

In [15]:
def get_price(game):
    #gets game price through parsing BS tree
    
    if game.select(".search_discount")[0].span:
        #price info can be placed in differend tags 
        #depends on discount ot not
        p = game.select(".discounted")[0].span.text
    else:
        p = game.select(".search_price")[0].text.strip()
    if p == '':
        return -1.0
    elif "free" in p.lower():
        return 0.0
    else:
        p = p.strip().replace(',', '.')
        return float(p[:-4])

In [19]:
def get_games_info(s):
    p = 1
    url = f'https://store.steampowered.com/search/results/?query&start={p}&count=100&dynamic_data=&sort_by=_ASC&tags=3843&untags=4182&category1=998&category3=36%2C9%2C38%2C1&os=win&filter=globaltopsellers&infinite=1'
    p = s
    response = requests.get(url)
    y = json.loads(response.content.decode('utf-8'))
    tree = BeautifulSoup(y['results_html'], 'html.parser')
    games = tree.select(".search_result_row")

    games_info = []
    for game in games:        
        link = game.get('href')
        g = {'game id':link.split('/')[4],
            'title': game.select(".title")[0].string,
            'release_date': game.select(".search_released")[0].string,
            'price': get_price(game),
            'rating_inPercent':get_rating(game),
            'url': link
            }
        games_info.append(g)
        print(games_info)
    return games_info

In [ ]:
games_data = []
for i in [100, 200, 300]:
    new_g = get_games_info(100)
    games_data += new_g
    if len(games_data) in [100, 200, 300]:
        time.sleep(3)
    print(new_g)
print('LENGTH:', len(games_data))

In [ ]:
df = pd.DataFrame(games_data)
df

In [22]:
df.to_csv('steam_coops.csv', index=True)

In [3]:
games = pd.read_csv('steam_coops.csv', index_col=0)
games.head(10)

,game id,title,release_date,price,rating_inPercent,url
0,252490,Rust,"8 Feb, 2018",725.0,86.0,https://store.steampowered.com/app/252490/Rust...
1,381210,Dead by Daylight,"14 Jun, 2016",499.0,82.0,https://store.steampowered.com/app/381210/Dead...
2,1097150,Fall Guys: Ultimate Knockout,"3 Aug, 2020",465.0,83.0,https://store.steampowered.com/app/1097150/Fal...
3,239140,Dying Light,"27 May, 2021",1099.0,95.0,https://store.steampowered.com/app/239140/Dyin...
4,582660,Black Desert,"25 Jul, 2019",350.0,76.0,https://store.steampowered.com/app/582660/Blac...
5,594650,Hunt: Showdown,"27 Aug, 2019",1349.0,83.0,https://store.steampowered.com/app/594650/Hunt...
6,286160,Tabletop Simulator,"5 Jun, 2015",419.0,95.0,https://store.steampowered.com/app/286160/Tabl...
7,552990,World of Warships,"15 Nov, 2017",0.0,81.0,https://store.steampowered.com/app/552990/Worl...
8,1359090,Zero Hour,"12 Aug, 2020",289.0,82.0,https://store.steampowered.com/app/1359090/Zer...
9,581320,Insurgency: Sandstorm,"12 Dec, 2018",999.0,85.0,https://store.steampowered.com/app/581320/Insu...
